# 원본

In [1]:
import librosa
import xgboost as xgb
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import zipfile
import os

# 압축 파일의 경로를 지정합니다. 예를 들어 'archive.zip'이라고 가정합니다.
zip_path = '/content/drive/MyDrive/all_wav_test.zip'

# 압축을 해제할 폴더의 경로를 지정합니다.
extract_folder = '/content/data'

# 압축 파일명에서 확장자를 제거하여 압축 해제 폴더 내에 하위 폴더 이름을 생성합니다.
folder_name = os.path.splitext(os.path.basename(zip_path))[0]
extract_path = os.path.join(extract_folder, folder_name)

# 새 폴더를 생성합니다.
os.makedirs(extract_path, exist_ok=True)

# 압축을 해제합니다.
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)




In [3]:
# 압축 파일의 경로를 지정합니다. 예를 들어 'archive.zip'이라고 가정합니다.
zip_path = '/content/drive/MyDrive/voice_output -test.zip'

# 압축을 해제할 폴더의 경로를 지정합니다.
extract_folder = '/content/data'

# 압축 파일명에서 확장자를 제거하여 압축 해제 폴더 내에 하위 폴더 이름을 생성합니다.
folder_name = os.path.splitext(os.path.basename(zip_path))[0]
extract_path = os.path.join(extract_folder, folder_name)

# 새 폴더를 생성합니다.
os.makedirs(extract_path, exist_ok=True)

# 압축을 해제합니다.
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)




In [4]:
import json

In [5]:
!pip install resampy
import resampy


In [49]:
def extract_features(file_name):
    try:
        audio, sample_rate = librosa.load(file_name, res_type='kaiser_fast')
        mfccs = librosa.feature.mfcc(y=audio, sr=sample_rate, n_mfcc=40)
        mfccsscaled = np.mean(mfccs.T,axis=0)
    except FileNotFoundError:
        print(f"File not found: {file_name}")
        return None
    except Exception as e:
        print(f"Error encountered while parsing file: {file_name}, error: {str(e)}")
        return None
    return mfccsscaled

In [50]:
# JSON 파일 로딩 및 데이터 및 라벨 생성
with open('/root/Compare_models/test_dataset.json', 'r') as f:
    folder_label_mapping = json.load(f)
print("JSON file loaded successfully!")

file_paths = []
labels = []

# 각 폴더 및 라벨에 대해
for file_path, label in folder_label_mapping.items():
    # 파일 확장자 확인하여 wav 파일만 처리
    if file_path.lower().endswith('.wav'):
        # 파일 경로 및 라벨 저장
        file_paths.append(file_path)
        labels.append(label)


JSON file loaded successfully!


In [51]:
# 특성 추출
features = []
new_labels = []  # 새로운 라벨 리스트를 만듭니다.
for file_name, label in zip(file_paths, labels):
    data = extract_features(file_name)
    if data is not None:  # 데이터가 None이 아닌 경우에만 추가
        features.append(data)
        new_labels.append(label)  # 해당 라벨도 추가합니다.

# 리스트를 NumPy 배열로 변환
features = np.array(features)
labels = new_labels  # 새로운 라벨 리스트를 사용합니다.




KeyboardInterrupt: 

In [10]:
# 2. 데이터 분할
X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.2, random_state=42)


In [12]:
# 3. XGBoost 모델 훈련

model = xgb.XGBClassifier(tree_method='hist', device='cuda:0')
model.fit(X_train, y_train)



Accuracy: 99.63%


In [ ]:

# 4. 예측 및 정확도 계산

predictions = model.predict(X_test)
accuracy = accuracy_score(y_test, predictions)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

# Validation

In [1]:
import librosa
import json
import math
import numpy as np
from tqdm import tqdm
import xgboost as xgb

from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical, Sequence
from sklearn.metrics import accuracy_score
from skimage.transform import resize



class Dataloader(Sequence):
    def __init__(self, Audios, labels, batch_size):
        self.Audios = Audios
        self.labels = labels
        self.batch_size = batch_size
        self.num_classes = len(set(self.labels))
        self.indices = np.arange(len(self.labels))

    def __len__(self):
        return math.ceil(len(self.labels)/self.batch_size)
    
    def __getitem__(self, idx):
        indices = self.indices[idx*self.batch_size : (idx+1)*self.batch_size]
        batch_x = [self.Audios[i] for i in indices]
        batch_audios = self.get_Audios(batch_x)
        batch_y = [self.labels[i] for i in indices]
        # 라벨을 원-핫 인코딩
        batch_y = to_categorical(batch_y, num_classes=self.num_classes)
        return np.array(batch_audios), np.array(batch_y)
    
    def get_Audios(self, path_list):
        # 오디오 데이터 로딩 및 전처리
        spectrograms = []
        for file_path in path_list:
            audio, sample_rate = librosa.load(file_path, res_type='kaiser_fast')
            mfccs = librosa.feature.mfcc(y=audio, sr=sample_rate, n_mfcc=40)
            mfccsscaled = np.mean(mfccs.T,axis=0)
            
            spectrograms.append(mfccsscaled) 

        return np.array(spectrograms)
    


# JSON 파일 로딩 및 데이터 및 라벨 생성
with open('../test_dataset.json', 'r') as f:
    folder_label_mapping = json.load(f)

file_paths = []
labels = []

# 각 폴더 및 라벨에 대해
for file_path, label in folder_label_mapping.items():
    # 파일 확장자 확인하여 wav 파일만 처리
    if file_path.lower().endswith('.wav'):
        # 파일 경로 및 라벨 저장
        file_paths.append(file_path)
        labels.append(label)

# 라벨을 정수로 변환 (만약 문자열 라벨을 사용하고 있다면)
unique_labels = sorted(set(labels))
label_to_int = {label: i for i, label in enumerate(unique_labels)}
labels = [label_to_int[label] for label in labels]

# 라벨의 종류 수 계산
num_classes = len(set(labels))

# 라벨 배열이 비어 있지 않은지 확인
if len(labels) == 0:
    raise ValueError("Labels array is empty. Check your data loading logic.") 


Validation_dataloader = Dataloader(file_paths, labels, 16)


2023-12-14 06:44:57.516719: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-12-14 06:44:57.516813: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-12-14 06:44:57.516855: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-12-14 06:44:57.540230: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:


epochs = 30

# 모델 불러오기
model_path = './model/xgboost_50epochs_model_sum.json'
model = xgb.XGBClassifier() # 모델 초기화




In [3]:
model.load_model(model_path) # 모델 불러오기

In [4]:
pred_list = []
label_list = []
#4. 예측 및 정확도 계산
for i in tqdm(range(Validation_dataloader.__len__())):
    x, y = Validation_dataloader.__getitem__(i)
    pred = model.predict(x)
    pred_list.append(pred)
    label_list.append(y)



100%|██████████| 601/601 [14:24<00:00,  1.44s/it]


In [5]:
def list_concat(x):
    temp = []
    for i in x:
        if len(temp) > 0:
            np.concatenate((temp, i))
        else:
            temp = i
    return temp

pred_list_concat = list_concat(pred_list)
label_list_concat = list_concat(label_list)
pred_list_concat = to_categorical(np.argmax(pred_list_concat, axis=1))
label_list_concat = to_categorical(np.argmax(label_list_concat, axis=1))

In [6]:
accuracy = accuracy_score(label_list_concat, pred_list_concat)
print("Accuracy: %.2f%%" % (accuracy * 100.0))


Accuracy: 68.75%


In [7]:
 from sklearn.metrics import f1_score

f1 = f1_score(label_list_concat, pred_list_concat, average='macro')
print("f1_score: %.2f%%" % (f1 * 100.0))



f1_score: 68.63%


In [8]:
from sklearn.metrics import log_loss
# 로그 손실 계산
loss = log_loss(label_list_concat, pred_list_concat)
print("loss_score: %.2f%%" % (loss))


loss_score: 4.98%
